In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
!cd

C:\Users\nkarlovic\erasmus\tox21\src\modules


## loading data:

In [3]:
data = pd.read_csv('../../data/tox21_final')
data = data.set_index('SMILES')
data.head()

,el0,el1,el2,el3,el4,el5,el6,el7,el8,el9,...,eccentricity,inertialshapefactor,npr1,npr2,pmi1,pmi2,pmi3,radiusofgyration,spherocityindex,calcpdf
SMILES,,,,,,,,,,,,,,,,,,,,,
CC(O)(P(=O)(O)O)P(=O)(O)O,6.477364,2.359698,-1.966891,10.177376,5.724987,-2.785115,9.932944,6.079196,3.075493,11.269686,...,0.883409,0.002830,0.468602,0.964442,340.837818,701.487124,727.350539,2.114170,0.574563,0.758799
O=S(=O)(Cl)c1ccccc1,3.053593,0.980627,0.760429,5.569807,2.050269,-2.193739,4.352890,2.075266,1.287445,4.511387,...,0.947976,0.004129,0.318343,0.984751,238.472771,737.683805,749.106857,2.242200,0.289602,0.392654
CC(C)COC(=O)C(C)C,4.030059,1.950864,2.656193,12.033220,3.177294,-3.923885,8.498935,3.130291,3.159135,10.303667,...,0.979349,0.007183,0.202179,0.949177,132.150073,620.408629,653.627755,2.342910,0.204214,0.439499
C=C(C)C(=O)OCCOC(=O)C(=C)C,5.590338,2.395565,2.317506,12.454545,4.090345,-4.186896,9.917704,4.153928,3.539044,11.350695,...,0.995883,0.005087,0.090653,0.960566,188.835938,2000.914227,2083.057362,3.406495,0.067441,0.396117
O=C([O-])Cc1cccc2ccccc12,3.988345,3.032919,3.969767,13.618371,4.071826,-5.485389,5.525861,3.777478,4.796931,8.252877,...,0.931774,0.001911,0.363039,0.735222,384.739517,779.168802,1059.773808,2.512492,0.130973,0.380954


In [4]:
data = data.replace({np.inf:0, -np.inf:0, np.nan:0, None:0})
data

,el0,el1,el2,el3,el4,el5,el6,el7,el8,el9,...,eccentricity,inertialshapefactor,npr1,npr2,pmi1,pmi2,pmi3,radiusofgyration,spherocityindex,calcpdf
SMILES,,,,,,,,,,,,,,,,,,,,,
CC(O)(P(=O)(O)O)P(=O)(O)O,6.477364,2.359698,-1.966891,10.177376,5.724987,-2.785115,9.932944,6.079196,3.075493,11.269686,...,0.883409,0.002830,0.468602,0.964442,340.837818,701.487124,727.350539,2.114170,0.574563,0.758799
O=S(=O)(Cl)c1ccccc1,3.053593,0.980627,0.760429,5.569807,2.050269,-2.193739,4.352890,2.075266,1.287445,4.511387,...,0.947976,0.004129,0.318343,0.984751,238.472771,737.683805,749.106857,2.242200,0.289602,0.392654
CC(C)COC(=O)C(C)C,4.030059,1.950864,2.656193,12.033220,3.177294,-3.923885,8.498935,3.130291,3.159135,10.303667,...,0.979349,0.007183,0.202179,0.949177,132.150073,620.408629,653.627755,2.342910,0.204214,0.439499
C=C(C)C(=O)OCCOC(=O)C(=C)C,5.590338,2.395565,2.317506,12.454545,4.090345,-4.186896,9.917704,4.153928,3.539044,11.350695,...,0.995883,0.005087,0.090653,0.960566,188.835938,2000.914227,2083.057362,3.406495,0.067441,0.396117
O=C([O-])Cc1cccc2ccccc12,3.988345,3.032919,3.969767,13.618371,4.071826,-5.485389,5.525861,3.777478,4.796931,8.252877,...,0.931774,0.001911,0.363039,0.735222,384.739517,779.168802,1059.773808,2.512492,0.130973,0.380954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCC1CO1,2.796873,1.998565,2.842565,9.708484,2.916008,-4.102632,3.166319,2.617044,3.420562,5.813919,...,0.949572,0.027199,0.313549,0.912543,33.550895,97.645619,107.003816,1.363705,0.296459,0.341729
FC(F)OC(F)(F)C(F)Cl,5.754533,2.003050,1.942544,10.459002,5.560658,-4.701877,7.097816,5.327867,4.294031,12.080736,...,0.909379,0.002891,0.415968,0.909277,314.543298,687.569936,756.172165,2.194967,0.451664,0.640266
ClCC1CO1,3.186995,2.122568,2.852847,9.547714,3.406363,-4.405299,3.244281,3.186561,3.883216,5.937068,...,0.967535,0.019476,0.252738,0.878908,45.127932,156.934715,178.556525,1.474903,0.264894,0.345581


In [5]:
scaler = MinMaxScaler()
data_sc = scaler.fit_transform(data)
data_sc = pd.DataFrame(data_sc, index=data.index, columns=data.columns)
data_sc

,el0,el1,el2,el3,el4,el5,el6,el7,el8,el9,...,eccentricity,inertialshapefactor,npr1,npr2,pmi1,pmi2,pmi3,radiusofgyration,spherocityindex,calcpdf
SMILES,,,,,,,,,,,,,,,,,,,,,
CC(O)(P(=O)(O)O)P(=O)(O)O,0.371341,0.246013,0.122546,0.328382,0.299562,0.509052,0.391754,0.325501,0.420508,0.309471,...,0.883409,2.319265e-06,0.474803,0.964442,0.012741,0.008634,0.008658,0.235310,0.582833,0.449080
O=S(=O)(Cl)c1ccccc1,0.175060,0.102236,0.322288,0.179715,0.107281,0.536200,0.171677,0.111117,0.329087,0.123885,...,0.947976,3.384619e-06,0.322556,0.984751,0.008914,0.009080,0.008917,0.249560,0.293771,0.232385
CC(C)COC(=O)C(C)C,0.231040,0.203389,0.461130,0.388262,0.166253,0.456776,0.335197,0.167607,0.424784,0.282943,...,0.979349,5.887109e-06,0.204855,0.949177,0.004940,0.007636,0.007780,0.260769,0.207154,0.260109
C=C(C)C(=O)OCCOC(=O)C(=C)C,0.320489,0.249752,0.436325,0.401857,0.214029,0.444702,0.391153,0.222416,0.444209,0.311695,...,0.995883,4.169315e-06,0.091853,0.960566,0.007059,0.024629,0.024795,0.379148,0.068412,0.234434
O=C([O-])Cc1cccc2ccccc12,0.228648,0.316200,0.557333,0.439409,0.213060,0.385093,0.217939,0.202259,0.508523,0.226628,...,0.931774,1.566295e-06,0.367844,0.735222,0.014382,0.009591,0.012615,0.279644,0.132858,0.225460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCC1CO1,0.160342,0.208362,0.474779,0.313253,0.152581,0.448570,0.124879,0.140126,0.438151,0.159653,...,0.949572,2.229315e-05,0.317698,0.912543,0.001254,0.001202,0.001274,0.151782,0.300727,0.202246
FC(F)OC(F)(F)C(F)Cl,0.329902,0.208830,0.408864,0.337469,0.290963,0.421061,0.279937,0.285272,0.482810,0.331743,...,0.909379,2.369397e-06,0.421473,0.909277,0.011758,0.008463,0.009001,0.244303,0.458165,0.378929
ClCC1CO1,0.182707,0.221291,0.475532,0.308065,0.178239,0.434676,0.127954,0.170620,0.461806,0.163035,...,0.967535,1.596320e-05,0.256082,0.878908,0.001687,0.001932,0.002125,0.164159,0.268707,0.204526


In [6]:
df = pd.read_csv('../../data/Tox21.csv')
df = df.set_index('SMILES')
df = df.drop('Molecule', axis=1)
df

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
SMILES,,,,,,,,,,,,
CC(O)(P(=O)(O)O)P(=O)(O)O,0,0,0,0,0,0,0,0,0,0,0,0
O=S(=O)(Cl)c1ccccc1,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)COC(=O)C(C)C,0,0,0,0,0,0,0,0,0,0,0,0
C=C(C)C(=O)OCCOC(=O)C(=C)C,0,0,0,0,0,0,0,0,0,0,0,0
O=C([O-])Cc1cccc2ccccc12,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
CCC1CO1,0,0,0,0,1,0,0,0,0,0,0,0
FC(F)OC(F)(F)C(F)Cl,0,0,0,0,0,0,0,0,0,0,0,0
ClCC1CO1,0,0,0,0,0,0,0,0,0,0,0,0


## train-test split:

In [394]:
features = data_sc
targets = df[['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER',
       'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP',
       'SR-p53']]

In [395]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, targets, test_size=0.2, random_state=1)

In [396]:
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
Y_train = torch.tensor(Y_train.values)
Y_test = torch.tensor(Y_test.values)

In [397]:
display(X_train.shape)
display(X_test.shape)
display(Y_train.shape)
display(Y_test.shape)

torch.Size([2463, 4330])

torch.Size([616, 4330])

torch.Size([2463, 12])

torch.Size([616, 12])

## model:

In [398]:
#model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size1, output_size):
        super(NeuralNet, self).__init__()
        # input to first hidden layer
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.sig1 = nn.Sigmoid() # first activation function
        # second hidden layer
        self.l2 = nn.Linear(hidden_size, hidden_size1)
        self.sig2 = nn.Sigmoid() # second activation function
        # third hidden layer and output
        self.l3 = nn.Linear(hidden_size1, output_size)
        self.soft = nn.Softmax(dim=1)


    def forward(self, x):
        # input to first hidden layer
        out = self.l1(x)
        out = self.sig1(out)
        # second hidden layer
        out = self.l2(out)
        out = self.sig2(out)
        # output layer
        out = self.l3(out)
        out = self.soft(out)
        return out
# init i forward are methods
# init creates layers, activation functions etc
# forward is called with the inputs (x), and returns the result (out)

In [399]:
model = NeuralNet(4330, 150, 80, 12)

In [400]:
print(model.parameters)

<bound method Module.parameters of NeuralNet(
  (l1): Linear(in_features=4330, out_features=150, bias=True)
  (sig1): Sigmoid()
  (l2): Linear(in_features=150, out_features=80, bias=True)
  (sig2): Sigmoid()
  (l3): Linear(in_features=80, out_features=12, bias=True)
  (soft): Softmax(dim=1)
)>


## training model:

In [409]:
# train the model
def train_model(model, X_tr, y_tr):
    # define the optimization
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    # enumerate epochs
    for epoch in range(900):
        # clear the gradients
        optimizer.zero_grad()
        # compute the model output
        y_pred = model(X_tr)
        # calculate loss
        loss = criterion(y_pred, y_tr)
        # credit assignment
        loss.backward()
        # update model weights
        optimizer.step()

In [410]:
train_model(model=model, X_tr=X_train.float(), y_tr=Y_train.float())

## predict:

In [411]:
y_pred = model(X_test.float())
y_pred = pd.DataFrame(y_pred.detach().numpy(), columns=targets.columns).round().astype(int)
y_pred

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
611,0,0,0,0,0,0,0,0,0,0,0,0
612,0,0,0,0,0,0,0,0,0,0,0,0
613,0,0,0,0,0,0,0,0,0,0,0,0
614,0,0,0,0,0,0,0,0,0,0,0,0


## accuracy:

In [412]:
Y_test_df = pd.DataFrame(Y_test.detach().numpy(), columns=targets.columns)
Y_test_df

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
611,0,0,0,0,0,0,0,0,0,0,0,0
612,0,0,0,0,0,0,0,1,0,0,0,0
613,0,0,0,0,0,0,0,0,0,0,0,0
614,0,0,1,0,0,0,0,0,0,0,0,0


In [413]:
for i in targets.columns:
    print(balanced_accuracy_score(Y_test_df[i], y_pred[i]))

0.5800546448087431
0.49508196721311476
0.553668890236507
0.49916943521594687
0.5041952707856598
0.5650027731558513
0.5
0.5279667422524565
0.5
0.5300055463117027
0.5316268486916951
0.5


In [378]:
mean_squared_error(Y_test_df, y_pred)

0.04518398268398269

In [379]:
r2_score(Y_test_df, y_pred)

-0.3072016076962962